In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [ ]:
api_key=''
channel_ids = ['UCtnHxTn2yi8Fs3R7EnuYKBA','UCsTknd_ddiwJ7_qwVPHv1XA','UCKcukj-n3-kUJ5u59L-bwgg','UCk19iYDCu8aV_KTDdELiucw']

youtube = build('youtube', 'v3', developerKey=api_key) #connecting youtube service

# channel statics

In [ ]:
def get_channel_stats(youtube,channel_id):
    all_data=[]
    
    request = youtube.channels().list(
            part="snippet,contentDetails,statistics",
            #id=",".join(channel_ids)
            id=",".join(channel_ids))
    response=request.execute()
    
    
#if we have the multiple channel we have to use loop
# GIVEN i there to visit each channel

    for i in range(len(response['items'])):
        data = dict(Channel_name = response['items'][i]['snippet']['title'],
                Subscribers = response['items'][i]['statistics']['subscriberCount'],
                Views = response['items'][i]['statistics']['viewCount'],
                Total_videos =response['items'][i]['statistics']['videoCount'],
                playlist_id =response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    return all_data
           
    

In [ ]:
channel_stat=get_channel_stats(youtube,channel_ids)

In [ ]:
channel_data=pd.DataFrame(channel_stat)
channel_data

# function to get vedio ids

In [ ]:
def get_video_ids(youtube, playlist_id):
    video_id = []
    
    # First request
    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=50
    )
    response = request.execute()

    for item in response['items']:
        video_id.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        
        # --- FIX 1: STOP AT 1000 TO SAVE QUOTA ---
        if len(video_id) >= 1000:
            break
        
        request = youtube.playlistItems().list(
            part='contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response['items']:
            video_id.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

    # --- FIX 2: RETURN THE FIRST 1000 ---
    return video_id[:1000] 


# --- RUNNING THE LOOP ---
all_video_ids = []

for i in range(len(channel_data)):
    playlist_id = channel_data.loc[i, 'playlist_id'] 
    ids = get_video_ids(youtube, playlist_id)
    print(f"Channel {i}: Found {len(ids)} videos")
    all_video_ids.append(ids)

# --- CRITICAL STEP: FLATTEN THE LIST ---
# If you don't do this, your next step will fail again!
flat_video_ids = [id for sublist in all_video_ids for id in sublist]

print(f"Total videos collected: {len(flat_video_ids)}")

In [ ]:
video_id=all_video_ids

# function to get vedio details

In [ ]:
def get_video_details(youtube, video_id):
    all_video_stats = []
    for i in range(0, len(video_id), 50):
        request = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=','.join(video_id[i:i+50])
        )
        response = request.execute()
        
        for video in response['items']:
            stats = video.get('statistics', {})
            snippet = video.get('snippet', {})
            content = video.get('contentDetails', {})

            video_stats = {
                'video_id': video['id'],  # <--- FIX: Use video['id'], NOT the video_id list
                'channel_name': snippet.get('channelTitle'),
                'title': snippet.get('title'),
                'subscribers': stats.get('subscriberCount'),
                'published_date': snippet.get('publishedAt'),
                'views': int(stats.get('viewCount', 0)),
                'likes': int(stats.get('likeCount', 0)),
                'comments': int(stats.get('commentCount', 0)),
                'duration': content.get('duration'),
                'tags': str(snippet.get('tags', [])), # Convert list to string for SQL
                'thumbnail': snippet.get('thumbnails', {}).get('high', {}).get('url')
            }
            all_video_stats.append(video_stats)
    return all_video_stats

In [ ]:
#checking the vedio length

flat_video_ids = [item for sublist in all_video_ids for item in sublist]
print(f"Total videos to fetch: {len(flat_video_ids)}") 
vedio_details = get_video_details(youtube, flat_video_ids)

In [ ]:
vedio_data=vedio_details

In [ ]:
edtech_data=pd.DataFrame(vedio_data)

In [ ]:
edtech_data.columns = [col.lower() for col in edtech_data.columns]

In [ ]:
edtech_data['channel_name'].value_counts()